In [ ]:
# The rest of the code needs this package update

!pip install datasets==4.0.0

import datasets
print(datasets.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get data from file

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Postdoc/cleaned_stripped_mimic_notes.csv")
print(df.shape)

(51695, 4)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for BERT-Tiny
tokenizer = AutoTokenizer.from_pretrained("gaunernst/bert-tiny-uncased")

from datasets import Dataset

# Convert pandas dataframe to HuggingFace dataset
dataset = Dataset.from_pandas(df)

# Tokenize all examples
def tokenize_batch(example):
    return tokenizer(
        example["clean_relevant_note_truncate"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/51695 [00:00<?, ? examples/s]

In [ ]:
# Reformat data
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

# Split data into train and temp
temp_split = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = temp_split["train"]
temp_dataset = temp_split["test"]

# Then split temp into val and test
val_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

In [ ]:
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/Postdoc/results/student/no-teacher-tiny/v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/Postdoc/logs/student/no-teacher-tiny/v2",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [ ]:
import numpy as np
!pip install evaluate
import evaluate

# Metrics to compute
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification

# Import pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(
    "gaunernst/bert-tiny-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at gaunernst/bert-tiny-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import torch

# Function to evaluate the model on the test set
def evaluate_model_on_test_set(trainer, test_dataset):
    # Ensure test dataset is in torch format
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Run model predictions
    predictions = trainer.predict(test_dataset)
    logits = predictions.predictions
    labels = predictions.label_ids
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1)
    preds = torch.argmax(probs, axis=1).numpy()

    # Compute metrics
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    auc = roc_auc_score(labels, probs[:, 1])
    cm = confusion_matrix(labels, preds)

    # Display results
    print("🔍 Test Set Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUROC:     {auc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    print("\nDetailed Classification Report:")
    print(classification_report(labels, preds, digits=4))

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm
    }

In [ ]:
import torch
from transformers import Trainer

# Custom class to give label = 0 or 1 different weights in the cross entropy loss function, to account for class imbalance
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Setting the weights here
        weights = torch.tensor([0.72, 1.60]).to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-12-1845219473.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


In [ ]:
# Run the model evaluation before training. As expected, accuracy and F1 are low.

results = evaluate_model_on_test_set(trainer, test_dataset)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lili-zeng (lili-zeng-mcgill-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


🔍 Test Set Evaluation:
Accuracy:  0.5020
Precision: 0.3142
Recall:    0.5209
F1 Score:  0.3920
AUROC:     0.5089

Confusion Matrix:
[[2648 2717]
 [1145 1245]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.6981    0.4936    0.5783      5365
           1     0.3142    0.5209    0.3920      2390

    accuracy                         0.5020      7755
   macro avg     0.5062    0.5072    0.4851      7755
weighted avg     0.5798    0.5020    0.5209      7755



In [ ]:
# Train model on 10 epochs
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Recall,Precision
1,0.558100,0.543312,0.001300,0.772762,0.615620,0.589390,0.644292
2,0.494500,0.533465,0.001300,0.787207,0.626359,0.577694,0.683976
3,0.455700,0.532216,0.001300,0.774568,0.635530,0.636591,0.634471
4,0.501700,0.517995,0.001300,0.782693,0.641108,0.628655,0.654063
5,0.483800,0.531495,0.001300,0.763090,0.640016,0.682122,0.602805
6,0.494900,0.529537,0.001300,0.783596,0.647183,0.642857,0.651566
7,0.507500,0.539371,0.001300,0.781790,0.648817,0.652882,0.644802
8,0.455900,0.535697,0.001300,0.779727,0.646377,0.652047,0.640805
9,0.415100,0.555860,0.001300,0.787980,0.640577,0.611947,0.672018
10,0.416300,0.555753,0.001300,0.784885,0.643894,0.629908,0.658515


TrainOutput(global_step=45240, training_loss=0.48662509846539753, metrics={'train_runtime': 931.172, 'train_samples_per_second': 388.607, 'train_steps_per_second': 48.584, 'total_flos': 459738440294400.0, 'train_loss': 0.48662509846539753, 'epoch': 10.0})

In [ ]:
# Save model and tokenizer to a folder
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2")           # saves model weights + config
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2")    # saves tokenizer files

('/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-no-teacher/v2/tokenizer.json')

In [ ]:
# Evaluate model again after training.
results = evaluate_model_on_test_set(trainer, test_dataset)

🔍 Test Set Evaluation:
Accuracy:  0.7831
Precision: 0.6476
Recall:    0.6498
F1 Score:  0.6487
AUROC:     0.8266

Confusion Matrix:
[[4520  845]
 [ 837 1553]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8438    0.8425    0.8431      5365
           1     0.6476    0.6498    0.6487      2390

    accuracy                         0.7831      7755
   macro avg     0.7457    0.7461    0.7459      7755
weighted avg     0.7833    0.7831    0.7832      7755



In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Optional code to find threshold that gives best F1 score

# Get raw logits from trainer.predict
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
labels = predictions.label_ids

# Try a range of thresholds
best_f1 = 0
best_thresh = 0.5

for thresh in np.arange(0.1, 0.9, 0.01):
    preds_thresh = (probs >= thresh).astype(int)
    _, _, f1, _ = precision_recall_fscore_support(labels, preds_thresh, average='binary')
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = thresh

print(f"Best F1: {best_f1:.4f} at threshold: {best_thresh:.2f}")

Best F1: 0.6528 at threshold: 0.46
